local:

In [50]:
import itertools
import numpy as np

def matrix(a, b, match_score=1, gap_cost=1):
    H = np.zeros((len(a) + 1, len(b) + 1), np.int)

    for i, j in itertools.product(range(1, H.shape[0]), range(1, H.shape[1])):
        match = H[i - 1, j - 1] + (match_score if a[i - 1] == b[j - 1] else - match_score)
        delete = H[i - 1, j] - gap_cost
        insert = H[i, j - 1] - gap_cost
        H[i, j] = max(match, delete, insert, 0)
    return H

def traceback(H, b, b_='', old_i=0):
    # flip H to get index of **last** occurrence of H.max() with np.argmax()
    H_flip = np.flip(np.flip(H, 0), 1)
    i_, j_ = np.unravel_index(H_flip.argmax(), H_flip.shape)
    i, j = np.subtract(H.shape, (i_ + 1, j_ + 1))  # (i, j) are **last** indexes of H.max()
    if H[i, j] == 0:
        return b_, i, j
    b_ = b[j - 1] + '-' + b_ if old_i - i > 1 else b[j - 1] + b_
    return traceback(H[0:i, 0:j], b, b_, i)

def local_alignment(a, b):
    H = matrix(a, b)
    b_, i, j = traceback(H, b)
    matches = len(b_.replace("-", ""))
    b_ = b[:j]+b_+b[j+matches:]
    if j > i:
        a = a.rjust(j-i+len(a), '-').ljust(len(b_), '-')
        b_ = b_.ljust(len(a), '-')
    else :
        b_ = b_.rjust(i-j+len(b_), '-').ljust(len(a), '-')
        a = a.ljust(len(b_), '-')
    print(a)
    print(b_)

local_alignment('BBBCCCBBBBBBAAAAAAABBBCCCC', 'BBBCCCCCCCCCCAAAAAABBBCCCC')

BBBCCCBBBBBBAAAAAAABBBCCCC-
BBBCCC-ABBBCCCCAAAAABBBCCCC


global:

In [49]:
%%bash
./t weights2.csv BBBCCCBBBBBBAAAAAAABBBCCCC BBBCCCCCCCCCCAAAAAABBBCCCC

weights:
map[AA:1 AB:-1 AC:-1 A_:-1 BA:-1 BB:1 BC:-1 B_:-1 CA:-1 CB:-1 CC:1 C_:-1 _A:-1 _B:-1 _C:-1]
aligned sequences:
BBBCCCBBBBBBAAAAAAABBBCCCC
BBBCCCCCCCCCCAAAAAABBBCCCC


with affine gap:

In [57]:
from pprint import pprint

global MIN
MIN = -float("inf")

#return match or mismatch score
def _match(s, t, i, j, match, mismatch):
    if t[i-1] == s[j-1]:
        return match
    else:
        return mismatch

#initializers for matrices
def _init_x(i, j):
    if i > 0 and j == 0:
        return MIN
    else:
        if j > 0:
            return -10 + (-0.5 * j)
        else:
            return 0

def _init_y(i, j):
    if j > 0 and i == 0:
        return MIN
    else:
        if i > 0:
            return -10 + (-0.5 * i)
        else:
            return 0

def _init_m(i, j):
    if j == 0 and i == 0:
        return 0
    else:
        if j == 0 or i == 0:
            return MIN
        else:
            return 0

def _format_tuple(inlist, i, j):
    return 0

def distance_matrix(s, t, match, mismatch, S, E):
    dim_i = len(t) + 1
    dim_j = len(s) + 1
    #abuse list comprehensions to create matrices
    X = [[_init_x(i, j) for j in range(0, dim_j)] for i in range(0, dim_i)]
    Y = [[_init_y(i, j) for j in range(0, dim_j)] for i in range(0, dim_i)]
    M = [[_init_m(i, j) for j in range(0, dim_j)] for i in range(0, dim_i)]

    for j in range(1, dim_j):
        for i in range(1, dim_i):
            X[i][j] = max((S + E + M[i][j-1]), (E + X[i][j-1]), (S + E + Y[i][j-1]))
            Y[i][j] = max((S + E + M[i-1][j]), (S + E + X[i-1][j]), (E + Y[i-1][j]))
            M[i][j] = max(_match(s, t, i, j, match, mismatch) + M[i-1][j-1], X[i][j], Y[i][j])

    return [X, Y, M]

def backtrace(s, t, X, Y, M, match, mismatch):
    sequ1 = ''
    sequ2 = ''
    i = len(t)
    j = len(s)
    while (i>0 or j>0):
        if (i>0 and j>0 and M[i][j] == M[i-1][j-1] + _match(s, t, i, j, match, mismatch)):
            sequ1 += s[j-1]
            sequ2 += t[i-1]
            i -= 1; j -= 1
        elif (i>0 and M[i][j] == Y[i][j]):
            sequ1 += '_'
            sequ2 += t[i-1]
            i -= 1
        elif (j>0 and M[i][j] == X[i][j]):
            sequ1 += s[j-1]
            sequ2 += '_'
            j -= 1

    sequ1r = ' '.join([sequ1[j] for j in range(-1, -(len(sequ1)+1), -1)])
    sequ2r = ' '.join([sequ2[j] for j in range(-1, -(len(sequ2)+1), -1)])

    return [sequ1r, sequ2r]

def affine_gap_alignment(seq1, seq2, weight_match, weight_mismatch, open_gap_penalty, continue_gap_penalty):
    [X, Y, M] = distance_matrix(seq1, seq2, weight_match, weight_mismatch, open_gap_penalty, continue_gap_penalty)
    [str1, str2] = backtrace(seq1, seq2, X, Y, M, weight_match, weight_mismatch)
    print(str1)
    print(str2)

print('-1., -1.:\n')
affine_gap_alignment('TACGGGCCCGCTAC', 'TAGCCCTATCGGTCA', 1., -1., -1., -1.)
print('\n\n-5., -1.:\n')
affine_gap_alignment('TACGGGCCCGCTAC', 'TAGCCCTATCGGTCA', 1., -1., -5., -1.)

-1., -1.:

T A C G G G C C _ _ C G C T A C
T A _ G C C C T A T C G G T C A


-5., -1.:

T A _ C G G G C C C G C T A C
T A G C C C T A T C G G T C A
